# Identifying bugs from launchpad

From the way Launchpad works the flow for a ticket opening in launchpad is:
1. a ticket, stating a possible bug, is opened in launchpad and its status is New,
2. if the problem stated in the ticket is reproduced than the bug is confirmed as genuine and the ticket status passes from New to Confirmed,
3. only when a bug is confirmed, the status then changes from Confirmed to In Progress in the same time that an issue that will try to solve the bug is opened in gerrit.

Tracing this flow, which data's can be extracted from the launchpad database, we can automatically identify bugs...

In [2]:
# importing needed libraries and establishing a connection to the database
import MySQLdb
import pandas

def connect():
   user = "root"
   password = ""
   host = "localhost"
   db = "tickets"
   port=3306
   use_unicode=True

   try:
      db = MySQLdb.connect(user = user, passwd = password, db = db, port = port, use_unicode = use_unicode)
      return db, db.cursor()
   except:
      logging.error("Database connection error!")
      raise
        
def execute_query(connector, query):
   results = int (connector.execute(query))
   cont = 0
   if results > 0:
      result1 = connector.fetchall()
      return result1
   else:
      return []
    
db, cursor = connect()

In [6]:
# Extracting the tickets from launchpad that do follow the flow: New -> Confirmed -> In progress. 
# Will store these datas in a table for facilitating later use. 
# Note: this datas contain duplicated issue
cursor.execute(""" create table bugs as
select ie.id,
       ie.status, 
       ie.issue_id,
       i.issue,
       ie.title, 
       ie.description, 
       ie.web_link, 
       ie.date_created, 
       ie.date_confirmed, 
       ie.date_in_progress
from issues_ext_launchpad ie
inner join issues i
on ie.issue_id=i.id
where ie.date_confirmed != '' 
and ie.date_in_progress != ''
""")

/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:18: Warning: Incorrect datetime value: '' for column 'date_confirmed' at row 1
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:18: Warning: Incorrect datetime value: '' for column 'date_in_progress' at row 1


57720L

In [17]:
# let's see how many of these are distinct issues
cursor.execute(""" select distinct issue from bugs """)

55061L

# Tickets analysed in Cinder

In [32]:
# from the table ComparationCinder, provided by Gema, let's extract the issues that have been classified as bugs, 
# and can be found in the table bugs. 
# Note: the query selects distinct issues from table bugs, whereas the table ComparationCinder contains duplicated 
# TicketIds, that will distinct subsequently.
cursor.execute(""" select distinct b.issue
from bugs b
inner join ComparationCinder c
on b.issue=c.TicketID
where c.Classification='Bug' """)

96L

In [21]:
# Now, lets extract the distinct issues from the table ComparationCinder that have been classified as bugs 
cursor.execute(""" select distinct TicketId from ComparationCinder where Classification='Bug' """)

96L

### Results: 
We see from the results that all the tickets classified as bugs from ComparationCinder are found in table bugs.

In [33]:
# from the table ComparationCinder, provided by Gema, let's extract the issues that have been classified as not bugs, 
# and check wether they can be found or not in the table bugs. 
# Note: the query selects distinct issues from table bugs, whereas the table ComparationCinder contains duplicated 
# TicketIds, that will distinct subsequently.
cursor.execute(""" select distinct b.issue
from bugs b
inner join ComparationCinder c
on b.issue=c.TicketID
where c.Classification='Not_Bug' """)

47L

In [23]:
# Now, lets extract the distinct issues from the table ComparationCinder that have been classified as not bugs 
cursor.execute(""" select distinct TicketId from ComparationCinder where Classification='Not_Bug' """)

47L

Results:

We see from the results that all the tickets classified as not bugs from ComparationCinder are found in table bugs.

In [34]:
# from the table ComparationCinder, provided by Gema, let's extract the issues that have been classified as Unknown, 
# and check wether they can be found or not in the table bugs. 
# Note: the query selects distinct issues from table bugs, whereas the table ComparationCinder contains duplicated 
# TicketIds, that will distinct subsequently.
cursor.execute(""" select distinct b.issue
from bugs b
inner join ComparationCinder c
on b.issue=c.TicketID
where c.Classification='Unknown' """)

11L

In [27]:
# Now, lets extract the distinct issues from the table ComparationCinder that have been classified as Unknown
cursor.execute(""" select distinct TicketId from ComparationCinder where Classification='Unknown' """)

11L

Results:

We see from the results that all the tickets classified as Unknown from ComparationCinder are found in table bugs.

# Tickets analysed in Horizon

In [31]:
# from the table ComparationCinder, provided by Gema, let's extract the issues that have been classified as bugs, 
# and can be found in the table bugs. 
# Note: the query selects distinct issues from table bugs, whereas the table ComparationCinder contains duplicated 
# TicketIds, that will distinct subsequently.
cursor.execute(""" select distinct b.issue
from bugs b
inner join ComparationHorizon h
on b.issue=h.TicketID
where h.Classification='Bug' """)

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 29))



OperationalError: (1412, 'Table definition has changed, please retry transaction')